In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
#Source Code for data Processing
#https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
#train = train_data.append(test_data)
#np.shape(train)

In [5]:
item_metadata = pd.read_csv('item_metadata.csv', sep=',', engine='python')

In [6]:
train_v2 = train_data[train_data['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [7]:
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [8]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int32
action_type        object
reference           int32
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [9]:
train_v2.head()
#np.shape(train_v2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [10]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

(1586586, 12)

In [11]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

,user_id,reference
13,00RL8Z82B2Z1,109038
15,00RL8Z82B2Z1,1257342
115,02SRUT1NQYH1,2795374
121,03K8AXBL4BX2,1032816
122,03K8AXBL4BX2,1032816


In [12]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

,user,item,click_count
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [13]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

,user,user_id
0,0001VQMGUI65,0
1,0003QTCX5MJX,1
2,0004IOZI7CKF,2
3,0004WCFRV3FB,3
4,0006W0R5A5V8,4


In [14]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

,item,item_id
0,477811,0
1,950829,1
2,2019467,2
3,3133074,3
4,2195060,4


In [15]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


# Side information

In [16]:
train = train_data
train['action_type'].unique()

array(['search for poi', 'interaction item image', 'clickout item',
       'interaction item info', 'interaction item deals',
       'search for destination', 'filter selection',
       'interaction item rating', 'search for item',
       'change of sort order'], dtype=object)

In [17]:
search_for_destination_table = train[train['action_type']=='search for destination']
search_for_destination_table = search_for_destination_table[['user_id','action_type']]
search_for_destination_table['search_for_destination'] = 1

change_of_sort_order_table = train[train['action_type']=='change of sort order']
change_of_sort_order_table = change_of_sort_order_table[['user_id','action_type']]
change_of_sort_order_table['change_of_sort_order'] = 1

filter_selection_table = train[train['action_type']=='filter selection']
filter_selection_table = filter_selection_table[['user_id','action_type']]
filter_selection_table['filter_selection'] = 1

interaction_item_image_table = train[train['action_type']=='interaction item image']
interaction_item_image_table = interaction_item_image_table[['user_id','action_type']]
interaction_item_image_table['interaction_item_image'] = 1

interaction_item_info_table = train[train['action_type']=='interaction item info']
interaction_item_info_table = interaction_item_info_table[['user_id','action_type']]
interaction_item_info_table['interaction_item_info'] = 1

interaction_item_deals_table = train[train['action_type']=='interaction item deals']
interaction_item_deals_table = interaction_item_deals_table[['user_id','action_type']]
interaction_item_deals_table['interaction_item_deals'] = 1

search_for_item_table = train[train['action_type']=='search for item']
search_for_item_table = search_for_item_table[['user_id','action_type']]
search_for_item_table['search_for_item'] = 1

search_for_poi_table = train[train['action_type']=='search for poi']
search_for_poi_table = search_for_poi_table[['user_id','action_type']]
search_for_poi_table['search_for_poi'] = 1

In [18]:
search_for_destination_table=search_for_destination_table.groupby('user_id').search_for_destination.nunique().reset_index()
search_for_destination_table.columns = ['user', 'search_for_destination_table']
change_of_sort_order_table=change_of_sort_order_table.groupby('user_id').change_of_sort_order.nunique().reset_index()
change_of_sort_order_table.columns = ['user', 'change_of_sort_order_table']
filter_selection_table=filter_selection_table.groupby('user_id').filter_selection.nunique().reset_index()
filter_selection_table.columns = ['user', 'filter_selection_table']
interaction_item_image_table=interaction_item_image_table.groupby('user_id').interaction_item_image.nunique().reset_index()
interaction_item_image_table.columns = ['user', 'interaction_item_image_table']
interaction_item_info_table=interaction_item_info_table.groupby('user_id').interaction_item_info.nunique().reset_index()
interaction_item_info_table.columns = ['user', 'interaction_item_info_table']
interaction_item_deals_table=interaction_item_deals_table.groupby('user_id').interaction_item_deals.nunique().reset_index()
interaction_item_deals_table.columns = ['user', 'interaction_item_deals_table']
search_for_item_table=search_for_item_table.groupby('user_id').search_for_item.nunique().reset_index()
search_for_item_table.columns = ['user', 'search_for_item_table']
search_for_poi_table=search_for_poi_table.groupby('user_id').search_for_poi.nunique().reset_index()
search_for_poi_table.columns = ['user', 'search_for_poi_table']
np.shape(search_for_poi_table)

(86650, 2)

In [19]:
search_for_poi_table.head()

,user,search_for_poi_table
0,0008BO33KUQ0,1
1,001JFV0AYE6Z,1
2,0022E2NEUGQX,1
3,0025B8BU0NYP,1
4,002B0QBQXZ62,1


In [20]:
list(interaction_item_image_table)

['user', 'interaction_item_image_table']

In [21]:
side_information_table = user_id_table.merge(search_for_destination_table, how='left', left_on='user', right_on = 'user').merge(change_of_sort_order_table, how='left', left_on='user', right_on = 'user').merge(filter_selection_table, how='left', left_on='user', right_on = 'user').merge(interaction_item_image_table, how='left', left_on='user', right_on = 'user').merge(interaction_item_info_table, how='left', left_on='user', right_on = 'user').merge(interaction_item_deals_table, how='left', left_on='user', right_on = 'user').merge(search_for_item_table, how='left', left_on='user', right_on = 'user').merge(search_for_poi_table, how='left', left_on='user', right_on = 'user')
side_information_table = side_information_table.fillna(0)
side_information_table = side_information_table.drop("user", axis=1)
side_information_table.head()
side_information_table.tail()

,user_id,search_for_destination_table,change_of_sort_order_table,filter_selection_table,interaction_item_image_table,interaction_item_info_table,interaction_item_deals_table,search_for_item_table,search_for_poi_table
717769,717769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717770,717770,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
717771,717771,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
717772,717772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717773,717773,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Basic Matrix Factorization

In [46]:
#Based on Albert Au Yeung Notes on machine learning and A.I.
class MF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            #if (i+1) % 10 == 0:
                #print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [55]:
G = sparse.csr_matrix((data['click_count'].astype(float), (data['user_id'], data['item_id'])))#make user/item graph
G = G[:5000,:5000]
G = G.todense()
#adjust alpha to and iterations for different matrix size 0.1 gives problems for 5000 rows but 0.01 works
mf = MF(G, K=5, alpha=0.01, beta=0.01, iterations=20)
training_process = mf.train()
users = list()
rating = dict()
for i in range(50):
    rating = dict()
    for j in range(50):
        rating[j] = mf.get_rating(i,j)
    users.append(rating)

#prints item ranking for user 0
d_view = sorted( ((v,k) for k,v in users[0].items()), reverse=True) 
for v,k in d_view:
    print(k,v)

48 1.9621506371226334
45 1.6210718261790324
11 1.4855314684385936
26 1.4783487585999504
27 1.4266639739057347
12 1.4055201775606356
3 1.3738261614398477
41 1.3638526705633165
9 1.3137397610443908
14 1.3013704034108424
10 1.2999826711643605
32 1.2950875189331361
19 1.2786179936153312
42 1.2764565290352885
0 1.2718920353838763
16 1.271767268857879
43 1.2713093358793826
15 1.25984024482086
34 1.2551286429672164
38 1.2536395255694532
47 1.2434170746863922
35 1.2398856922021446
30 1.237050128799212
28 1.2287221051538746
8 1.2231367460151468
44 1.2182092600321803
29 1.2014616297414176
40 1.1977694918774247
20 1.193996127954479
39 1.1927966317334504
36 1.1867156171890487
23 1.180417470384425
1 1.1730534069340368
22 1.1721807808692586
6 1.1716389560297171
13 1.1645743578496452
37 1.1611840083737741
4 1.1519077752009164
2 1.143181336817459
24 1.1422212629657584
33 1.1350484443429951
31 1.133998906411889
18 1.1245477966990038
7 1.110420108681531
25 1.086230720159181
49 1.0815128534965415
17 1.05

# Graph Embedding

In [22]:
#Author Manuel Aguero
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse as sparse
import numpy as np
import random
import mpmath

def mini_batch_generator(graph,batch_size,row_length,col_length):
    i = random.randint(0, row_length-batch_size)
    j = random.randint(0,col_length-batch_size)
    return graph[i:i+batch_size,j:j+batch_size]

def negative_sampling(E,row_length,col_length,batch_size):
    S = sparse.dok_matrix((row_length+1,col_length+1)) #create empty negtive feedack graph
    for i in range(batch_size):
        a = random.randint(0, row_length)
        b = random.randint(0,col_length)
        randomEdge = (a,b) #randomly sample edges from the implicit feedback graph
        if (a,b) not in E:
            S[a,b]=1
    return S.tocoo()

def compute_e_ife(theta_u,theta_v,G_items,S_items):
    sumG = sumS = 0
    for (i,j) in G_items:
        sumG += np.log(1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j])))))
    for (i,j) in S_items:
        sumS += np.log(1 - (1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j]))))) )
    return sumG + sumS
            
def update_embeddings(theta_u,theta_v,G_items,G_mini_items,S_items,a,e_ife):
    sumGu = sumSu = sumGv = sumSv = 0
    for (i,j) in G_mini_items:
        currVal = (1 - 1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j])))))
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumGu += uVal
        sumGv += vVal
    for (i,j) in S_items:
        currVal = (-1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j])))))
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumSu += uVal
        sumSv += vVal
    theta_u = theta_u + (a*sumGu + a*sumSu) #update user embeddings
    theta_v = theta_v + (a*sumGv + a*sumSv) #update item embeddings
    new_eife = compute_e_ife(theta_u,theta_v,G_items,S_items) #check S: expand S or use new mini S?
    difference = new_eife - e_ife
    #print("new, prev",new_eife,e_ife)
    #print(difference)
    e_ife=new_eife
    if e_ife > new_eife:
        theta_u = theta_u - (a*sumGu + a*sumSu) #undo
        theta_v = theta_v - (a*sumGv + a*sumSv) #undo
    else:
        e_ife = new_eife
    return (difference,e_ife,theta_u,theta_v)
    
def implicit_feedback_embedding(G,b,a,k,row_length,col_length):
    #initiliase list of vector embeddings randomly
    theta_u = list()
    theta_v = list()
    for i in range(row_length+1):
        theta_u.append(np.random.rand(k))
    for i in range(col_length+1):
        theta_v.append(np.random.rand(k))   
    tolerance = 10 #convergence parameter epsilon
    convergence_difference = 100 #arbitrary to initialisation
    G_items = set(zip(G.row,G.col)) #get iterator for row,col pair with non-zero values
    e_ife = None
    G = G.tocsr()
    #print(theta_u[0],theta_v[0])
    while abs(convergence_difference) > tolerance: #while embedding has not converged
        #print("diff > tolerance?",abs(convergence_difference),tolerance)
        G_mini = mini_batch_generator(G,b,row_length,col_length)
        S = negative_sampling(G_items,row_length,col_length,b) #make negative feedback graph
        G_mini = G.tocoo()
        S_items = set(zip(S.row,S.col))
        G_mini_items = set(zip(G_mini.row,G_mini.col))
        #print("p",theta_u[0])
        if not e_ife:
            e_ife = compute_e_ife(theta_u,theta_v,G_items,S_items) #initialise e_ife mini G or G?
        convergence_difference,e_ife,theta_u,theta_v= update_embeddings(theta_u,theta_v,G_items,G_mini_items,S_items,a,e_ife)
        #print("n",theta_u[0])
        #print(e_ife)
    #print("diff > tolerance?",convergence_difference,tolerance)
    return (theta_u,theta_v)

def build_information_pool(theta_u,theta_v,S):
    I_u = list()
    for i in range(len(theta_u)):
        I_u.append(np.concatenate((theta_u[i],S[i])))
    I_u = np.array(I_u)
        
def deep_matrix_factorization(G,Si):
    print("Start DMF...\n")
    Si = S[:5000,:5000]
    G = G[:5000,:5000]
    G = G.tocoo()
    row_length = max(G.row)
    col_length = max(G.col)
    batch_size = 200
    learning_rate = 0.0001
    dimensionality = 5
    print("Implicit feedback embedding...")
    theta_u,theta_v = implicit_feedback_embedding(G,batch_size,learning_rate,dimensionality,row_length,col_length)
    print("embedding done")
    print("build info pools...")
    build_information_pool(theta_u,theta_v,Si)
    print("pools done")
    
     
G = sparse.csr_matrix((data['click_count'].astype(float), (data['user_id'], data['item_id'])))#make user/item graph
S = side_information_table.values
#deep_matrix_factorization(G,S)

#print("\nDMF done")


DMF done
